In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable
from torch.utils.data import Dataset,DataLoader
import time
import numpy as np
import matplotlib.pyplot as plt
from datasets import *
from Pix2pix_model import *

In [2]:
saved_dir="../saved_data/pix2pix_facades/"
sample_dir="../samples/pix2pix_facades/"
# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)
    print("folders created!")
else:
    print("folders not created!")
if not os.path.exists(saved_dir):
    os.makedirs(saved_dir)
    print("folders created!")
else:
    print("folders not created!")

folders not created!
folders not created!


In [3]:
batch_size=16
n_epochs=201
image_width=256
learning_rate=2e-4

In [4]:
transforms_ = transforms.Compose([transforms.ToTensor(),
                                  transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
train_data=ImageDataset(root='D:/pix2pix_dataset/facades/train/',
                        transform=transforms.ToTensor())
train_loader=torch.utils.data.DataLoader(train_data,
                                        batch_size=batch_size,
                                        shuffle=True)

In [5]:

generator_model=Generator().cuda()
discriminator_model=Discriminator().cuda()

optimizer_G=torch.optim.Adam(generator_model.parameters(),lr=learning_rate)
optimizer_D=torch.optim.Adam(discriminator_model.parameters(),lr=learning_rate)
gen_loss=nn.MSELoss().cuda()
l1_loss=nn.L1Loss().cuda()

In [7]:
start=time.time()
total_step=len(train_loader)
for epochs in range(n_epochs):
    for i ,image in enumerate(train_loader):
        image_num=int(image['A'].shape[0])
        y_image=image['A']
        x_image=image['B']
        
        
        true_label=torch.ones(image_num,1,8,8).cuda() 
        false_label=torch.zeros(image_num,1,8,8).cuda()


        x_image=Variable(x_image).cuda()
        y_image=Variable(y_image).cuda()
        #------------------
        #Generator training
        #------------------
        optimizer_G.zero_grad()
        gen_out=generator_model.forward(x_image)
#         print("gen_out.shape : ",gen_out.shape,"reshaped_x_image.shape:",reshaped_x_image.shape)
        gen_dis_out=discriminator_model(gen_out,x_image)

#         print("gen_dis_out.shape : ",gen_dis_out.shape)
        g_loss=gen_loss(gen_dis_out,true_label)
        g_l1=l1_loss(gen_out,y_image)

        loss_G=g_loss+g_l1*100
#         loss_G=g_loss
        loss_G.backward(retain_graph=True)
        optimizer_G.step()


        #----------------------
        #Discriminator training
        #----------------------

        optimizer_D.zero_grad()
        #d fake loss
        dis_fake_out=discriminator_model.forward(gen_out,y_image)
        d_fake_loss=gen_loss(dis_fake_out,false_label)

        #d real loss
        dis_real_out=discriminator_model.forward(x_image,y_image)
        d_real_loss=gen_loss(dis_real_out,true_label)

        loss_D=(d_fake_loss+d_real_loss)*0.5
        loss_D.backward()
        optimizer_D.step()
        
#     if epochs==0:
#         x_image.cpu()
#         y_image.cpu()
        
#         xx=x_image.detach()
#         yy=y_image.detach()
        
#         save_image(yy,
#                    os.path.join(sample_dir,'true_Y_images.png'))
#         save_image(xx,
#                    os.path.join(sample_dir,'true_X_images.png'))
    save_image(gen_out,os.path.join(
        sample_dir,'fake_images-{}.png'.format(epochs+1)))
    print ("[Epoch %d/%d] [D loss: %f] [G loss: %f]" % (epochs, n_epochs,
                                                        loss_D.item(), loss_G.item()))
    if(epochs%50==0):
        # Save the model checkpoints
        torch.save(generator_model.state_dict(), saved_dir +
                   '/G_pix2pix_{}.ckpt'.format(epochs+1))
        torch.save(discriminator_model.state_dict(), saved_dir +
                   '/D_pix2pix_{}.ckpt'.format(epochs+1))
finished = time.time()
hours = finished-start
print("training finished! %d minutes" % hours)       

C:\Users\nhs\Anaconda3\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


[Epoch 0/201] [D loss: 0.065930] [G loss: 19.880947]
[Epoch 1/201] [D loss: 0.022466] [G loss: 18.433662]
[Epoch 2/201] [D loss: 0.009712] [G loss: 17.936781]
[Epoch 3/201] [D loss: 0.008439] [G loss: 18.879578]
[Epoch 4/201] [D loss: 0.005083] [G loss: 18.339052]
[Epoch 5/201] [D loss: 0.004214] [G loss: 15.622816]
[Epoch 6/201] [D loss: 0.004549] [G loss: 16.037148]
[Epoch 7/201] [D loss: 0.004531] [G loss: 15.426266]
[Epoch 8/201] [D loss: 0.007108] [G loss: 16.150074]
[Epoch 9/201] [D loss: 0.002883] [G loss: 15.287408]
[Epoch 10/201] [D loss: 0.004550] [G loss: 15.545678]
[Epoch 11/201] [D loss: 0.003084] [G loss: 15.085279]
[Epoch 12/201] [D loss: 0.002941] [G loss: 15.747424]
[Epoch 13/201] [D loss: 0.003193] [G loss: 14.556160]
[Epoch 14/201] [D loss: 0.002834] [G loss: 13.119689]
[Epoch 15/201] [D loss: 0.002105] [G loss: 14.099004]
[Epoch 16/201] [D loss: 0.003271] [G loss: 13.743205]
[Epoch 17/201] [D loss: 0.001659] [G loss: 13.009998]
[Epoch 18/201] [D loss: 0.001450] [G l

[Epoch 154/201] [D loss: 0.000222] [G loss: 5.541589]
[Epoch 155/201] [D loss: 0.000234] [G loss: 5.336924]
[Epoch 156/201] [D loss: 0.000474] [G loss: 5.303771]
[Epoch 157/201] [D loss: 0.000242] [G loss: 5.004519]
[Epoch 158/201] [D loss: 0.000373] [G loss: 4.624597]
[Epoch 159/201] [D loss: 0.000212] [G loss: 4.851966]
[Epoch 160/201] [D loss: 0.000231] [G loss: 5.176715]
[Epoch 161/201] [D loss: 0.000151] [G loss: 5.004280]
[Epoch 162/201] [D loss: 0.000154] [G loss: 4.925781]
[Epoch 163/201] [D loss: 0.000207] [G loss: 4.784417]
[Epoch 164/201] [D loss: 0.000549] [G loss: 4.627234]
[Epoch 165/201] [D loss: 0.000265] [G loss: 4.967001]
[Epoch 166/201] [D loss: 0.000342] [G loss: 5.037655]
[Epoch 167/201] [D loss: 0.000302] [G loss: 5.007478]
[Epoch 168/201] [D loss: 0.000212] [G loss: 4.960209]
[Epoch 169/201] [D loss: 0.000220] [G loss: 4.936253]
[Epoch 170/201] [D loss: 0.000570] [G loss: 5.171854]
[Epoch 171/201] [D loss: 0.000208] [G loss: 4.821294]
[Epoch 172/201] [D loss: 0.0